
# Grammar Scoring from Voice Samples — Colab (Pipeline A, Audio Mode 3)

This Colab notebook runs the full pipeline end-to-end:

- Clone your GitHub repo
- Install dependencies (quietly)
- Auto-detect audio in `data/sample_audio/` **or** let you upload your own
- Transcribe with Whisper **small** (faster-whisper)
- Extract grammar errors, fillers, WPM
- Score & explain (0–100)
- Visualize component penalties + overall score gauge
- Optional batch evaluation if multiple files are provided

> **Model:** `small` (balanced speed/accuracy)  
> **Audio Mode 3:** auto-detect sample **or** upload on the fly


In [ ]:

#@title Runtime & Config
import os, sys, platform, subprocess, shutil, json
from pathlib import Path

print("Python:", sys.version)
print("OS:", platform.platform())
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))
print("GPU available (nvidia-smi):", shutil.which("nvidia-smi") is not None)
if shutil.which("nvidia-smi"):
    _ = subprocess.run(["nvidia-smi", "-L"], check=False)

# Configuration (edit if needed)
REPO_URL = "https://github.com/bharathbattu/Grammar-Scoring-from-Voice-Samples"  #@param {type:"string"}
BRANCH = "main"  #@param {type:"string"}
MODEL_SIZE = "small"  #@param ["tiny", "small", "medium"]
PROJECT_DIR = Path("/content/Grammar-Scoring-from-Voice-Samples")
APP_DIR = PROJECT_DIR / "app"
DATA_DIR = PROJECT_DIR / "data" / "sample_audio"
print("Repo:", REPO_URL)
print("Branch:", BRANCH)
print("Model size:", MODEL_SIZE)


In [ ]:

#@title Clone/refresh repository
import shutil
from pathlib import Path
import subprocess

# Clean any prior clone
if Path(PROJECT_DIR).exists():
    print("Removing existing project directory...")
    shutil.rmtree(PROJECT_DIR, ignore_errors=True)

print("Cloning repo...")
_ = subprocess.run(
    ["git", "clone", "--depth", "1", "--branch", BRANCH, REPO_URL, str(PROJECT_DIR)],
    check=True
)
print("Done.")
print("Project files:", list(PROJECT_DIR.iterdir())[:8])


In [ ]:

#@title Install Dependencies (quiet)
import subprocess, shutil, sys, os

def pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "--quiet"] + pkgs
    return subprocess.run(cmd, check=False)

# Torch (choose CUDA if GPU present; otherwise CPU)
if shutil.which("nvidia-smi"):
    print("Installing Torch with CUDA wheels... (this may take a minute)")
    pip_install(["torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu121"])
else:
    print("Installing CPU Torch wheels...")
    pip_install(["torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cpu"])

# Install requirements from repo (fastapi, faster-whisper, pydantic v1, etc.)
print("Installing requirements from repo...")
req_file = PROJECT_DIR / "requirements.txt"
if req_file.exists():
    pip_install(["-r", str(req_file)])
else:
    # minimal fallback (shouldn't happen since repo has requirements.txt)
    pip_install(["fastapi", "uvicorn[standard]", "pydantic<2", "faster-whisper", "language-tool-python", "jiwer", "numpy", "pandas", "matplotlib"])

# Ensure Java for language_tool_python (Colab often has it; install if missing)
if not shutil.which("java"):
    print("Installing Java (for LanguageTool)...")
    subprocess.run(["apt-get", "update", "-qq"], check=False)
    subprocess.run(["apt-get", "install", "-y", "openjdk-11-jre-headless"], check=False)
else:
    print("Java found.")

print(" Dependencies installed.")


In [ ]:

#@title Import modules
import sys
from pathlib import Path

# Add project to sys.path
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

# Imports from the repo
from app.asr import transcribe
from app.text_features import grammar_errors, filler_count, words_per_minute, normalize_transcript
from app.scoring import (
    normalize_grammar_errors, normalize_fillers, normalize_wer,
    fluency_penalty, calculate_final_score, generate_score_explanation
)

import pandas as pd
import matplotlib.pyplot as plt

print(" Repo modules imported.")
print("Project root:", PROJECT_DIR)


In [ ]:

#@title Audio selection (Auto-detect or Upload)
from pathlib import Path
from typing import List, Optional
from IPython.display import display, HTML

DATA_DIR.mkdir(parents=True, exist_ok=True)
print("Sample audio dir:", DATA_DIR)

# Auto-detect first .wav or .mp3
audio_files: List[Path] = list(DATA_DIR.glob("*.wav")) + list(DATA_DIR.glob("*.mp3"))
audio_file: Optional[Path] = None

if audio_files:
    audio_file = audio_files[0]
    print(f"✓ Detected audio file in repo: {audio_file.name}")
else:
    print("⚠ No audio found in repo. You can upload one now...")

# Offer upload if none detected (or if you want to override)
try:
    from google.colab import files  # type: ignore
    do_upload = False if audio_file else True  # upload only if none found by default
    if do_upload:
        print("Opening browser file picker...")
        uploaded = files.upload()
        for name, _ in uploaded.items():
            src = Path(name)
            target = DATA_DIR / src.name
            src.replace(target)
            audio_file = target
            print("Saved to:", target)
    else:
        print("Using auto-detected file. To upload instead, set do_upload=True in this cell and re-run.")
except Exception as e:
    print("Upload widget not available (non-Colab env). Using detected file if any.")

if audio_file and audio_file.exists():
    size_kb = audio_file.stat().st_size / 1024
    print(f"Selected: {audio_file.name}  ({size_kb:.2f} KB)")
else:
    print("No audio file available. The notebook will fall back to an example transcript.")


In [ ]:

#@title Optional: Waveform (if librosa available)
try:
    import librosa, librosa.display  # type: ignore
    import numpy as np
    if audio_file and audio_file.exists():
        y, sr = librosa.load(str(audio_file), sr=None)
        duration = librosa.get_duration(y=y, sr=sr)
        plt.figure(figsize=(12, 3))
        librosa.display.waveshow(y, sr=sr, alpha=0.7)
        plt.title(f"Waveform: {audio_file.name}")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.tight_layout()
        plt.show()
        print(f"Duration: {duration:.2f}s  |  Sample rate: {sr} Hz")
    else:
        print("No audio file to visualize.")
except Exception as e:
    print("Skipping waveform:", e)


In [ ]:

#@title Transcribe with Whisper (small)
from typing import Dict, Any

if audio_file and audio_file.exists():
    print("Transcribing... (first run may download model ~150MB)")
    asr_result: Dict[str, Any] = transcribe(str(audio_file), model_size=MODEL_SIZE)
else:
    print("No audio found. Using example transcript.")
    asr_result = {
        "transcript": (
            "Um, hello. My name is John and, you know, I am applying "
            "for this position. I has experience in data science."
        ),
        "word_count": 21,
        "duration_sec": 8.5,
        "language": "en"
    }

print("\n=== ASR RESULT ===")
print("Transcript:", asr_result["transcript"][:300] + ("..." if len(asr_result["transcript"])>300 else ""))
print("Word count:", asr_result["word_count"])
print("Duration (s):", asr_result["duration_sec"])
print("Language:", asr_result.get("language", "en"))


In [ ]:

#@title Feature Extraction
from typing import List, Dict, Any

transcript: str = asr_result["transcript"]
normalized_text: str = normalize_transcript(transcript)
word_count: int = asr_result["word_count"]
duration_sec: float = asr_result["duration_sec"]

g_count: int
g_details: List[Dict[str, Any]]
g_count, g_details = grammar_errors(normalized_text, language="en-US")

f_count: int
f_list: List[str]
f_count, f_list = filler_count(normalized_text)

wpm = words_per_minute(word_count, duration_sec)

print("=== FEATURES ===")
print("Grammar errors:", g_count)
print("Fillers:", f_count, "|", f_list[:10])
print("WPM:", wpm)

# Show a compact table of grammar issues (top 10)
if g_count > 0:
    import pandas as pd
    df = pd.DataFrame([
        {
            "Rule ID": e["rule_id"],
            "Message": e["message"],
            "Context": (e["context"][:60] + "...") if len(e["context"]) > 60 else e["context"],
            "Suggestions": ", ".join(e.get("replacements", [])[:2]) if e.get("replacements") else ""
        }
        for e in g_details[:10]
    ])
    from IPython.display import display
    display(df)
else:
    print("No grammar errors detected.")


In [ ]:

#@title Scoring
g_pen = normalize_grammar_errors(g_count, word_count)
f_pen = normalize_fillers(f_count, word_count)
wer_pen = normalize_wer(None)  # no reference in this notebook
fl_pen = fluency_penalty(wpm)

final_score = calculate_final_score(
    grammar_penalty=g_pen,
    filler_penalty=f_pen,
    wer_penalty=wer_pen,
    fluency_pen=fl_pen
)

explanation = generate_score_explanation(
    grammar_penalty=g_pen,
    filler_penalty=f_pen,
    wer_penalty=wer_pen,
    fluency_pen=fl_pen,
    final=final_score
)

print("=== PENALTIES (0=best,1=worst) ===")
print("Grammar:", round(g_pen, 3))
print("Fillers:", round(f_pen, 3))
print("WER:    ", round(wer_pen, 3))
print("Fluency:", round(fl_pen, 3))

print("\n=== FINAL SCORE ===")
print(final_score, "/ 100")
print(explanation)


In [ ]:

#@title Visualization
from matplotlib.patches import Circle
from matplotlib.figure import Figure
from matplotlib.axes import Axes
import matplotlib.pyplot as plt

categories = ["Grammar\n(35%)", "Fillers\n(25%)", "WER\n(20%)", "Fluency\n(20%)"]
penalties = [g_pen, f_pen, wer_pen, fl_pen]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot of penalties
colors = ['#e74c3c' if p > 0.5 else '#f39c12' if p > 0.25 else '#27ae60' for p in penalties]
ax1.barh(categories, penalties, color=colors, alpha=0.7)
ax1.set_xlabel('Penalty [0=Perfect, 1=Worst]')
ax1.set_title('Component Penalties')
ax1.set_xlim(0, 1)
ax1.grid(axis='x', alpha=0.3)

for i, p in enumerate(penalties):
    ax1.text(p + 0.02, i, f'{p:.3f}', va='center')

# Gauge-like overall score
ax2.axis('off')
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)

score_normalized = final_score / 100.0
if score_normalized >= 0.8:
    color = '#27ae60'; grade = 'Excellent'
elif score_normalized >= 0.6:
    color = '#f39c12'; grade = 'Good'
elif score_normalized >= 0.4:
    color = '#e67e22'; grade = 'Fair'
else:
    color = '#e74c3c'; grade = 'Needs Improvement'

circle = Circle((0.5, 0.5), 0.35, color=color, alpha=0.3)
ax2.add_patch(circle)
ax2.text(0.5, 0.6, f'{final_score:.1f}', ha='center', va='center', fontsize=48, fontweight='bold', color=color)
ax2.text(0.5, 0.4, 'out of 100', ha='center', va='center', fontsize=12, color='gray')
ax2.text(0.5, 0.25, grade, ha='center', va='center', fontsize=16, fontweight='bold', color=color)
ax2.set_title('Overall Proficiency Score', fontsize=14, pad=20)

plt.tight_layout()
plt.show()


In [ ]:

#@title Batch Evaluation (if multiple files exist)
from typing import Dict, Any, List
import pandas as pd

if DATA_DIR.exists():
    files = list(DATA_DIR.glob("*.wav")) + list(DATA_DIR.glob("*.mp3"))
    if len(files) > 1:
        print(f"Found {len(files)} files. Running batch...")
        results: List[Dict[str, Any]] = []
        for i, ap in enumerate(files, 1):
            try:
                asr = transcribe(str(ap), model_size=MODEL_SIZE)
                tnorm = normalize_transcript(asr["transcript"])
                g_c, _ = grammar_errors(tnorm, language="en-US")
                f_c, _ = filler_count(tnorm)
                wpm_val = words_per_minute(asr["word_count"], asr["duration_sec"])
                g_p = normalize_grammar_errors(g_c, asr["word_count"])
                f_p = normalize_fillers(f_c, asr["word_count"])
                fl_p = fluency_penalty(wpm_val)
                score = calculate_final_score(g_p, f_p, 0.0, fl_p)
                results.append({
                    "File": ap.name,
                    "Words": asr["word_count"],
                    "Duration (s)": asr["duration_sec"],
                    "Grammar Errors": g_c,
                    "Fillers": f_c,
                    "WPM": wpm_val,
                    "Score": score
                })
                print(f"[{i}/{len(files)}] {ap.name}: {score:.2f}")
            except Exception as e:
                print(f"[{i}/{len(files)}] {ap.name}: ERROR -> {e}")
        if results:
            df = pd.DataFrame(results).sort_values("Score", ascending=False)
            print("\n=== BATCH RESULTS ===")
            from IPython.display import display
            display(df)
            print("Average score:", round(df["Score"].mean(), 2))
    else:
        print("Only one audio file found. Add more files to enable batch evaluation.")
else:
    print("Sample audio directory not found.")


In [ ]:

#@title Model & Library Versions
import importlib

def safe_ver(mod_name):
    try:
        m = importlib.import_module(mod_name)
        return getattr(m, "__version__", "unknown")
    except Exception:
        return "not installed"

print("faster-whisper:", safe_ver("faster_whisper"))
print("language_tool_python:", safe_ver("language_tool_python"))
print("jiwer:", safe_ver("jiwer"))
print("pandas:", safe_ver("pandas"))
print("matplotlib:", safe_ver("matplotlib"))
print("torch:", safe_ver("torch"))
